In [14]:
import numpy as np
from scipy.optimize import minimize
from scipy import stats
import scipy.integrate as integrate

y = np.array([1505.0, 1528, 1564, 1498, 1600, 1470])
m0 = np.mean(y)
sigma = np.sqrt(500)
n = len(y)

def fun(tau0):
    p = stats.norm.pdf
    integrand = lambda theta: p(theta, m0, tau0) ** n * np.prod(p(y, theta, sigma))
    return integrate.quad(integrand, 1400, 1700)[0]

minimize(fun, 1, method = 'SLSQP', bounds = [[0.5, 2]])

  status: 0
 success: True
    njev: 1
    nfev: 3
     fun: 1.0460367218786729e-20
       x: array([ 1.])
 message: 'Optimization terminated successfully.'
     jac: array([  8.69136082e-20,   0.00000000e+00])
     nit: 1